In [6]:
'''第一步：生成数据集'''
from mxnet import autograd,nd

num_inputs = 2;#特征值
num_examples = 1000;#样本数量
true_w = [2,-3.4] #真实权重
true_b = 4.2 #真实偏差
#创建样本特征值
'''
ramdom:几种采样方法说明
（1）均匀分布采样   uniform
（2）正太分布采样   normal
（3）泊松分布采样   poisson

'''
features = nd.random.normal(scale=1,shape=(num_examples,num_inputs));
#创建标签
labels =  true_w[0] * features[:,0] + true_w[1] * features[:,1] + true_b;
#加上噪声
labels += nd.random.normal(scale=0.01,shape=labels.shape);



In [7]:
'''第二步：读取数据集'''
#Gluon提供了data包来读取数据
from mxnet.gluon import data as gdata;
#数据样本的小批量大小
batch_size = 10;
#将训练数据的特征和标签组合
dataset = gdata.ArrayDataset(features,labels);
#随机load小批量
data_iter = gdata.DataLoader(dataset,batch_size,shuffle=True);

#看一个例子
for X,y in data_iter:
    print(X,y)
    break



[[-1.4074123   2.6135678 ]
 [ 0.5195597   0.9168663 ]
 [ 1.0993656  -1.9022169 ]
 [ 1.3810002  -0.7629666 ]
 [ 1.2391903  -0.5411598 ]
 [-0.29296088 -0.30558026]
 [-1.085067    0.47845957]
 [-0.17266594 -1.1916859 ]
 [ 0.27309823 -0.6894807 ]
 [-0.14719068  1.5903426 ]]
<NDArray 10x2 @cpu(0)> 
[-7.511107   2.0938976 12.872027   9.539722   8.525438   4.6608973
  0.4070814  7.917271   7.0896244 -1.5003345]
<NDArray 10 @cpu(0)>


In [8]:
'''第三步：定义模型'''
#nn是neural networks（神经网络）的缩写，这里边定义了大量神经网络的层
from mxnet.gluon import nn;
#在Gluon中，Sequential实例可以看作是一个串联各个层的容器
net = nn.Sequential();
#作为一个单层神经网络，输出层和输入层完全连接，因此叫做全连接层
#在Gluon中，全连接层是一个Dense实例，我们定义该层输出个数为1
net.add(nn.Dense(1))

#PS:无需指定没一层输入的形状，例如线性会对的输入个数
#当模型得到数据时，例如后边执行net(X)时，模型将自动推断每一层的输入个数



In [9]:
'''第四步:初始化模型参数'''

#在使用net前，我们需要初始化模型参数，如线性回归模型中的权重和偏差
#我们使用mxnet中的init模块，用init.Normal(sigma=0.01)指定权重参数每个元素在初始化是
#随机采样于均值为0，标准差为0.01的正态分布。偏差参数默认会初始化为零

from mxnet import init;

net.initialize(init.Normal(sigma=0.01));


In [10]:
'''第五步：定义损失函数'''
#在Gluon中，loss模块定义了各种损失函数
from mxnet.gluon import loss as gloss;
#平方损失又称为L2范数损失
loss = gloss.L2Loss();


In [11]:
'''第六步：定义优化算法'''
#小批量随机梯度下降sgd
#创建一个Trainer实例，学习率指定为0.03，
#该优化算法将用来迭代net实例所有通过add函数嵌套的层，所包含的全部参数
#这些参数通过collect_params函数获取
from mxnet import gluon;
#设置一下优化算法
trainer = gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':0.03})


In [12]:
'''第七步：训练模型'''
num_epochs = 3;#三轮全样本训练
for epoch in range(1,num_epochs +1):
    for X,y in data_iter:
        with autograd.record():
            l = loss(net(X),y)
        l.backward()
        #上边计算出来的损失，在这里就能起到作用
        trainer.step(batch_size)
    #然后算出每一轮的损失
    l = loss(net(features),labels);
    print('epoch %d, loss: %f' % (epoch, l.mean().asnumpy()))




epoch 1, loss: 0.034993
epoch 2, loss: 0.000124
epoch 3, loss: 0.000048


In [13]:
#最后我们比较一下学到的模型参数和真实的模型参数。
#我们从net获得需要的层，并访问其权重weight和偏差bias

dense = net[0]
#先看下权重
print(true_w,dense.weight.data())
#再看下偏差
print(true_b,dense.bias.data())



[2, -3.4] 
[[ 1.9998659 -3.399725 ]]
<NDArray 1x2 @cpu(0)>
4.2 
[4.1999354]
<NDArray 1 @cpu(0)>
